## RGB Mirror Orientation

http://localhost:8888/?token=sloth

In [ ]:
from tqdm import trange

# import torch
import numpy as np
import matplotlib.pyplot as plt

import drjit as dr
import mitsuba as mi

from libs import utils

plt.style.use('ggplot')

mi.set_variant('cuda_ad_mono')
mi.set_log_level(mi.LogLevel.Warn)

sess_seed   = 0 # np.random.randint(0, 2**30)
sess_seed_g = 1 # np.random.randint(0, 2**30)
print(f"session seeds are: sess_seed={sess_seed}; sess_seed_g={sess_seed_g}")

### Scene Construction

In [ ]:
vis = True

tf          = mi.ScalarTransform4f
mic_pos     = np.array([ 0.,  0.,  5.])
speaker_pos = np.array([20.,  0.,  5 ])
sail_pose   = np.array([10., 20.,  5.])
sail_size   = np.array([8.,  2.,  5.]) / 2.

sail_vertex_base = mi.Transform4f.scale(sail_size).rotate([1., 0., 0.], angle=90.) @ mi.Point3f(
    [-1., -1.,  1.,  1.],
    [-1.,  1., -1.,  1.],
    [ 0.,  0.,  0.,  0.]
)

scene_dict = {
    "type": "scene",
    "integrator": {
        "type": "prb_reparam",
        "max_depth": 2,
        "border_warning": False,
    },

    "sensor": {
        "type": "perspective",
        "to_world": tf.look_at(
            origin=(mic_pos),
            target=(sail_pose),
            up=[0, 1, 0]
        ),
        "film": {
            "type": "hdrfilm",
            "width":  128 if vis else 1,
            "height": 128 if vis else 1,
            "sample_border": True,
            "filter": { "type": "tent", "radius": 1.0, },
            # "sample_border": False,
            # "filter": { "type": "box" },
            "pixel_format": "luminance",
        },
        # "fov": 60,
        "sampler": { "type": "stratified", "sample_count": 2**20, },
    },

    "speaker": {
        "type": "ply",
        "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/sphere.ply",
        "to_world": tf.translate(speaker_pos).scale(2.0),
        "emitter": { "type": "area", "radiance": { "type": "uniform", "value": 1e0 } },
    },

    "sail": {
        "type": "ply",
        "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/rectangle.ply",
        "bsdf": {
            # "type": "roughconductor",
            # "alpha": 0.05,

            "type": "acousticbsdf",
            "scattering": { "type": "spectrum", "value": 0.20 },
            "absorption": { "type": "spectrum", "value": 0.20 },

            # 'type': 'blendbsdf',
            # 'weight': 0.1,
            # 'bsdf_0': { 'type': 'conductor' },
            # 'bsdf_1': { 'type': 'diffuse' },
        },
    },
}

### Optimization Setup

In [ ]:
key = "sail.vertex_positions"

opt = mi.ad.Adam(lr=0.005)
opt['a'] = mi.Point2f(0.1, 0.1)

def apply_transform(params, opt):
    opt['a'] = dr.clamp(opt['a'], -2., 2.)
    transf = mi.Transform4f.translate(sail_pose) \
                           .rotate(axis=[1., 0., 0.], angle=(opt['a'].x * 30.)) \
                           .rotate(axis=[0., 0., 1.], angle=(opt['a'].y * 30.))
    params[key] = dr.ravel(transf @ sail_vertex_base)
    params.update()

### Load and Render Scene

In [ ]:
scene  = mi.load_dict(scene_dict)
params = mi.traverse(scene)
opt['a'] = mi.Point2f(0.0, 0.0)
apply_transform(params, opt)

if vis:
    img = mi.render(scene, seed=sess_seed, spp=1024)
    utils.plot_img(img)
else:
    img = mi.render(scene, seed=sess_seed)
    print(img.numpy())

### Loss Values

In [ ]:
opt['a'] = mi.Point2f(0.1, 0.1)
apply_transform(params, opt)
img = mi.render(scene, params, seed=sess_seed, seed_grad=sess_seed_g, spp=64 if vis else 2**22)
l = dr.mean(img)
dr.backward(l)

print(dr.grad(opt['a'])[0])

assert True

In [ ]:
angles = np.linspace(-0.5, 0.5, 51, endpoint=True)
data = np.zeros((angles.shape[0], 3))

for i in trange(angles.shape[0]):
    opt['a'] = mi.Point2f(0, angles[i])
    apply_transform(params, opt)
    img = mi.render(scene, params, seed=sess_seed+i, seed_grad=sess_seed_g+i, spp=64 if vis else 2**22)

    l   = dr.mean(img)
    dr.backward(l)

    data[i, 0] = l[0]
    data[i, 1:] = dr.grad(opt['a']).numpy()[0]

In [ ]:
i, j = np.sort(np.argsort(data[:, 0])[-2:])
p = angles[np.argmin(data[i:j, 0])]
# p = angles[np.argmax(data[:, 0])]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(angles, data[:, 0])
# ax1.vlines(p, data[:, 0].min(), data[:, 0].max(), linestyle='dotted', color="k")
ax1.set_xlabel("opt['a']")
ax1.set_ylabel("loss")

ax2.plot(angles, data[:, 2])
# ax2.vlines(p, data[:, 1].min(), data[:, 1].max(), linestyle='dashed', linewidth=1.0, color="k")
# ax2.hlines(0, angles[0], angles[-1], linestyle='dotted', color="k", linewidth=1.0)
ax2.set_xlabel("opt['a']")
ax2.set_ylabel("grad")
plt.show()

In [ ]:
heatmap = True

if heatmap:
    k = 0

    vA   = np.linspace(-0.35, 0.35, 29, endpoint=True)
    vB   = np.linspace(-0.45, 0.45, 37, endpoint=True)
    a, b = np.meshgrid(vA, vB)
    data = np.zeros((3,) + a.shape)
    print(vA)
    print(vB)

    for i in trange(a.shape[0]):
        for j in range(a.shape[1]):
            opt['a'] = mi.Point2f(a[i, j], b[i, j])
            apply_transform(params, opt)

            img = mi.render(scene, params, seed=sess_seed, seed_grad=sess_seed_g, spp=64 if vis else 2**22)
            l   = dr.mean(img)
            dr.backward(l)
            data[0, i, j] = l[0]
            data[1, i, j] = dr.grad(opt["a"]).x[0]
            data[2, i, j] = dr.grad(opt["a"]).y[0]

In [ ]:
# np.save("../data/ceiling-sail/mirror-angle-heatmap.npy", np.vstack([a[None, :, :], b[None, :, :], data]))

In [ ]:
if heatmap:
    n = 2
    fig, ax = plt.subplots(1, 1, figsize=(7, 6))

    # contf = ax.contourf(a[12:-12, 10:-10], b[12:-12, 10:-10], data[0, 12:-12, 10:-10], cmap="inferno")
    contf = ax.contourf(a, b, data[0], cmap="inferno", levels=np.linspace(0., 0.012, 25))
    # ax.contour(a, b, data[0], colors='k', alpha=0.5, linewidths=0.3)

    ax.quiver(a, b, data[1], data[2], color='C1')

    fig.colorbar(contf, ax=ax, fraction=0.1, shrink=0.8).set_label('loss')
    # ax.set_xticks(vA[::n], np.round(vA * 30., 1)[::n])
    # ax.set_yticks(vB[::n], np.round(vB * 30., 1)[::n])
    ax.set_xlabel("$R_x$")
    ax.set_ylabel("$R_z$")
    ax.set_title(f"Loss Heatmap - ceiling sail {k}")

    # fig.savefig(f"../data/ceiling-sail/ceiling-sail-single-mic-heatmap-sail_{k}.png")
    fig.show()

### Main Optimization Loop

In [ ]:
opt['a'] = mi.Float(0.0)
angles, losses, grads = [], [], []

In [ ]:
iters = 50

for i in trange(iters) if iters > 1 else range(iters):
    apply_transform(params, opt)
    img = mi.render(scene, params, seed=sess_seed+i, seed_grad=sess_seed_g+i, spp=64 if vis else 2**20)

    l = -1. * dr.sum(img)
    dr.backward(l, flags=dr.ADFlag.ClearNone if iters < 2 else dr.ADFlag.Default)

    if iters < 2:
        # display(dr.unravel(mi.Point3f, dr.grad(params[key])))
        display(dr.grad(opt['a']))
    else:
        grads.append(dr.grad(opt['a']))
        opt.step()
        angles.append(opt['a'])
        losses.append(l)

#np.save("/home/daniel/loss_n.npy", angles)

In [ ]:
if iters > 1:
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 4))
    ax1.set_title("grads")
    ax1.plot(np.array(grads)[:, 0])

    ax2.set_title("angles")
    ax2.plot(np.array(angles)[:, 0])

    ax3.set_title("losses")
    ax3.plot(np.array(losses)[:, 0])
    fig.show()